In [ ]:
import sys
print(sys.executable)
print(sys.version)

In [ ]:
!which pip

In [ ]:
!python3 -m pip install psycopg2

## Create the Beers table

```sql
CREATE TABLE beers (
    id         INT PRIMARY KEY,
    name       VARCHAR(100) NOT NULL,
    brewery    VARCHAR(100) NOT NULL,
    style      VARCHAR(50) NOT NULL,
    abv        DECIMAL(4, 2),
    ounces     INTEGER,
    valid_dt   TIMESTAMPTZ DEFAULT NOW(),
    created_dt TIMESTAMPTZ DEFAULT NOW()
);

CREATE INDEX ON BEERS(name) INCLUDE(style, abv, ounces);
```

## Generate 1m Random Beer Entries

```sql
WITH data AS (
  SELECT
    array['Golden', 'Dark', 'Crisp', 'Bold', 'Smooth', 'Frosty', 'Hoppy', 'Mellow', 'Fiery', 'Rich', 'Bitter', 'Tangy', 'Earthy', 'Spicy', 'Floral'] AS adj_list,
    array['Dragon', 'Eagle', 'Bear', 'River', 'Mountain', 'Wolf', 'Storm', 'Phoenix', 'Falcon', 'Oak', 'Thunder', 'Vine', 'Horizon', 'Bison', 'Canyon'] AS noun_list,
    array['BrewHound', 'Sierra Arizona', 'Flounders Brewing', 'Rock Brewing Co', 'Ding''s Brewery', 'Catfish Head', 'Luganitas Brewing', 'Duck Island'] AS brewery_list,
    array['Ale', 'Pale Ale', 'IPA', 'Porter', 'Stout', 'Lager', 'Pilsner', 'Wheat Beer', 'Saison'] AS style_list
),
arrays AS (
  SELECT
    array_length(data.adj_list,1)::int AS adj_len,
    array_length(data.noun_list,1)::int AS noun_len,
    array_length(data.brewery_list,1)::int AS brewery_len,
    array_length(data.style_list,1)::int AS style_len
  FROM data
)
INSERT INTO beers (id, name, brewery, style, abv, ounces)
SELECT
    i,
    (data.adj_list[ceil(random() * arrays.adj_len)] || ' ' || data.noun_list[ceil(random() * arrays.noun_len)]) AS name,
    (data.brewery_list[ceil(random() * arrays.brewery_len)]) AS brewery,
    (data.style_list[ceil(random() * arrays.style_len)]) AS style,
    round((random() * 9 + 4)::numeric, 2) AS abv,
    (array[8, 12, 16, 24, 32])[ceil(random() * 5)]::int AS ounces
FROM generate_series(1, 1000000) AS s(i), data, arrays;
```

## Confirm Data Distribution by Ounces

Before starting the test, verify the current distribution by ounces:

```sql
select ounces, count(ounces) from beers group by ounces order by ounces asc;
```

When the test starts, this query can be used as it will return the values from the index:

```sql
select ounces, count(ounces) from beers where ounces in (8, 12, 16, 24, 32) group by ounces order by ounces asc;
```

In [ ]:
import psycopg2
import random
import time
from concurrent.futures import ThreadPoolExecutor
from psycopg2 import pool

def update_random_row():
    try:
        conn = connection_pool.getconn()
        cursor = conn.cursor()
      
        cursor.execute("""
        UPDATE beers
        SET ounces = (ARRAY[8, 12, 16, 24, 32])[ceil(random()*5)]
        WHERE id = ceil(random()*1000000);
        """)
        conn.commit()
        
        sleep(random.uniform(.01, .1))
    except psycopg2.DatabaseError as e:
        print(f"Database error occurred: {e}")
        if conn is not None:
            conn.rollback()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if cursor is not None:
            cursor.close()
        if conn:
            connection_pool.putconn(conn)

start_time = time.time()

connection_pool = pool.SimpleConnectionPool(
    minconn=1,
    maxconn=20,
    host='localhost',
    port='5433',
    user='yugabyte',
    password='',
    database='yugabyte'
)

with ThreadPoolExecutor(max_workers=40) as executor:
    futures = [executor.submit(update_random_row) for _ in range(1000)]
    
for future in futures:
    future.result()

connection_pool.closeall()

elapsed_time = time.time() - start_time
print(f"Done in: {elapsed_time} seconds")